In [1]:
import os
import json
import requests
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime, timedelta

pd.get_option("display.max_columns")

20

In [344]:
url = 'https://www.autodiler.me/automobili/all/'

headers = {'User-Agent': 'Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0'}

for num in range(277, 279):  # <<< заменить на while True
    r = requests.get(url + str(num), headers=headers)
    if r.status_code != 200:
        print(f'status_code: {r.status_code}   >>>   page number: {num}')
    with open(f'/home/pavlk/Documents/python-projects/autodiler/pages/page-{num}.txt', 'wb') as f:
        f.write(r.content)

In [2]:
files_dir = os.path.join(os.getcwd(), 'pages')
files = os.scandir(files_dir)

In [345]:
for file in files:
    code = BeautifulSoup(open(os.path.join(files_dir, file), 'rb'), 'html.parser')

In [3]:
code = BeautifulSoup(open(os.path.join(files_dir, 'page-277.txt'), 'rb'), 'html.parser')

In [4]:
c = code.find_all('script')[1].string

In [10]:
processed = c[c.find('{'):c.find('module={')-1].strip()

In [11]:
src = json.loads(processed)

In [65]:
def print_tree(source, limit=0, n=0):
    if n == limit and limit:
        return
    for key in source:
        print("    " * n, n, "-", key)
        if isinstance(source[key], dict):
            print_tree(source[key], limit, n+1)
print_tree(src, 3)

 0 - props
     1 - store
     1 - isServer
     1 - initialState
         2 - user
         2 - categories
         2 - groups
         2 - ads
         2 - common
         2 - geoObjects
         2 - garage
         2 - discountCard
         2 - form
     1 - initialProps
         2 - pageProps
 0 - page
 0 - pathname
 0 - query
     1 - category
     1 - type
     1 - page
 0 - buildId
 0 - assetPrefix
 0 - nextExport
 0 - err
 0 - chunks


In [13]:
[t for t in src['props']['initialState']['categories']]

['elements',
 'bySeo',
 'currentCategory',
 'currentSubcategory',
 'currentParentCategoryId',
 'isCategoryPage',
 'filters',
 'sort',
 'specifications',
 'activeCategories',
 'similarWords']

In [402]:
[t for t in src['props']['initialState']['ads']]

['list',
 'detail',
 'filter',
 'pagination',
 'brands',
 'products',
 'equipment',
 'add',
 'listings',
 'services',
 'serviceTypes',
 'edit',
 'shared',
 'sms']

In [403]:
[t for t in src['props']['initialState']['ads']]

['list',
 'detail',
 'filter',
 'pagination',
 'brands',
 'products',
 'equipment',
 'add',
 'listings',
 'services',
 'serviceTypes',
 'edit',
 'shared',
 'sms']

In [404]:
[x for x in src['props']['initialState']['ads']['list']]

['elements', 'isLoading', 'paid', 'paidRequested']

In [405]:
data = pd.DataFrame(src['props']['initialState']['ads']['list']['elements'])

In [406]:
data.shape[0] == 0

False

In [407]:
data.dropna(subset=['code'], inplace=True)
data.drop(labels=['banner', 'labelDescription', 'compatibilities'], axis=1, inplace=True)

In [408]:
empty = data.count().loc[data.count()==0].index

In [409]:
data = data.drop(labels=empty, axis=1)

In [410]:
data['media'] = data['media'].apply(lambda x: x[0]['fd'])
data.sample()

,code,city,media,specifications,createdAt,price,listingStatus,listingExpiration,id,seo,...,oldPrice,categoryTitle,priceOnRequest,priceType,listingActivated,category,user,status,titleCompiled,activated
15,AD56610507ME,"{'country': 1, 'name': 'Podgorica', 'id': 10, ...",https://slike.autodiler.me/2020/12/15/AD566105...,"[{'measure': None, 'weightCatalog': 3, 'icon':...",2020-12-15T12:58:06.000Z,1800.0,None,None,339071.0,auto-u-dobrom-stanju-sitna-ulaganja-dore-zim-g...,...,NaN,Automobili,False,2.0,None,2.0,69639.0,Published,Audi - A4 - 2.5,2020-12-15T13:03:32.000Z


In [421]:
data.columns[10:20]

Index(['listing', 'brand', 'product', 'thumbnail', 'refreshed', 'oldPrice',
       'categoryTitle', 'priceOnRequest', 'priceType', 'listingActivated'],
      dtype='object')

In [422]:
def generate_adlink(row):
    return 'https://www.autodiler.me/' + row['categoryTitle'] + '/' + row['seo']

In [425]:
data['ad_link'] = data.apply(generate_adlink, axis=1)

In [432]:
[print(link) for link in data.ad_link]

https://www.autodiler.me/Automobili/golf-sportsvan-1-6-tdi-dsg-ad57265507me
https://www.autodiler.me/Automobili/volksvagen-buba-u-dobrom-stanju-ad57265229me
https://www.autodiler.me/Automobili/chevrolet-aveo-5dr-1-4-mt5-ad56887043me
https://www.autodiler.me/Automobili/jeep--cherokee-2-800-crd-presao-150-000--2004-godiste-ad56823770me
https://www.autodiler.me/Automobili/mali-potrosac--bez-ulaganja-ad56823061me
https://www.autodiler.me/Automobili/prodajem-jeep-cheeroke-u-odlicnom-stanju--ad56883460me
https://www.autodiler.me/Automobili/prodajem-auto-u-odlicnom-stanju--prvi-vlasnik--auto-uvezeno-iz-svajcarske--ad38884239me
https://www.autodiler.me/Automobili/prodajem-bmw-x5-ad56820400me
https://www.autodiler.me/Automobili/audi-2-4-ad56820215me
https://www.autodiler.me/Automobili/prodajem-peugot-206-ad56882629me
https://www.autodiler.me/Automobili/u-dobrom-stanju-ulozeno-2-000-e-ovlasceni-servis-grebovic-nove-zimske-gume-i-ljetnje--ad56818377me
https://www.autodiler.me/Automobili/peugeot-3

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [361]:
data['country'] = data['city'].apply(lambda x: x['country'])
data['region'] = data['city'].apply(lambda x: x['region'])
data['city_id'] = data['city'].apply(lambda x: x['id'])
data['city'] = data['city'].apply(lambda x: x['name'])

In [362]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 36
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code               36 non-null     object 
 1   city               36 non-null     object 
 2   media              36 non-null     object 
 3   specifications     36 non-null     object 
 4   createdAt          36 non-null     object 
 5   price              36 non-null     float64
 6   listingStatus      1 non-null      object 
 7   listingExpiration  1 non-null      object 
 8   id                 36 non-null     float64
 9   seo                36 non-null     object 
 10  listing            36 non-null     object 
 11  brand              36 non-null     object 
 12  product            36 non-null     object 
 13  thumbnail          36 non-null     object 
 14  refreshed          7 non-null      object 
 15  oldPrice           4 non-null      float64
 16  categoryTitle      36 non-nu

In [363]:
data['brand_id'] = data['brand'].apply(lambda x: x['id'])
data['brand'] = data['brand'].apply(lambda x: x['name'])

In [364]:
data['product_id'] = data['product'].apply(lambda x: x['id'])
data['series'] = data['product'].apply(lambda x: x['series'])
data['model'] = data['product'].apply(lambda x: x['model'])
data = data.drop('product', axis=1)

In [365]:
data = data.drop('thumbnail', axis=1)

In [366]:
for date_col in ['createdAt', 'listingExpiration', 'listingActivated', 'activated']:
    data[date_col] = pd.to_datetime(data[date_col], format='%Y-%m-%dT%H:%M:%S.000Z')
data[['createdAt', 'listingExpiration', 'listingActivated', 'activated']].head()

,createdAt,listingExpiration,listingActivated,activated
0,2020-12-16 08:54:53,NaT,NaT,2020-12-16 09:10:12
1,2020-12-16 08:05:34,NaT,NaT,2020-12-16 08:23:05
2,2020-12-15 22:13:05,NaT,NaT,2020-12-16 07:50:44
3,2020-12-15 19:05:11,NaT,NaT,2020-12-15 19:23:32
4,2020-12-15 18:45:47,NaT,NaT,2020-12-15 18:51:31


In [367]:
def get_fuel(spec_list):
    for spec in spec_list:
        if str(spec['title']).lower().startswith('gorivo'):
            return spec['value']
        
def get_year(spec_list):
    for spec in spec_list:
        if str(spec['title']).lower().startswith('godi'):
            return spec['value']

def get_km(spec_list):
    for spec in spec_list:
        if str(spec['title']).lower().startswith('kilometra'):
            return spec['value']

In [368]:
data['fuel'] = data['specifications'].apply(get_fuel)
data['km'] = data['specifications'].apply(get_km)
data['year'] = data['specifications'].apply(get_year)

In [369]:
data[['fuel', 'km', 'year']].head()

,fuel,km,year
0,Dizel,211000,2016
1,Benzin,0,1975
2,Benzin,38000,2013
3,Dizel,150000,2004
4,Benzin,240000,2001


In [370]:
data

,code,city,media,specifications,createdAt,price,listingStatus,listingExpiration,id,seo,...,country,region,city_id,brand_id,product_id,series,model,fuel,km,year
0,AD57265507ME,Podgorica,https://slike.autodiler.me/2020/12/16/AD572655...,"[{'measure': None, 'weightCatalog': 3, 'icon':...",2020-12-16 08:54:53,10600.0,None,NaT,343166.0,golf-sportsvan-1-6-tdi-dsg-ad57265507me,...,1,2,10,160,3031,Golf,Golf Sportsvan,Dizel,211000,2016
1,AD57265229ME,Podgorica,https://slike.autodiler.me/2020/12/16/AD572652...,"[{'measure': None, 'weightCatalog': 3, 'icon':...",2020-12-16 08:05:34,3500.0,None,NaT,343163.0,volksvagen-buba-u-dobrom-stanju-ad57265229me,...,1,2,10,160,3011,None,Beetle,Benzin,0,1975
2,AD56887043ME,Tivat,https://slike.autodiler.me/2020/12/15/AD568870...,"[{'measure': None, 'weightCatalog': 2, 'icon':...",2020-12-15 22:13:05,3800.0,None,NaT,341154.0,chevrolet-aveo-5dr-1-4-mt5-ad56887043me,...,1,1,5,37,561,None,Aveo,Benzin,38000,2013
3,AD56823770ME,Podgorica,https://slike.autodiler.me/2020/12/15/AD568237...,"[{'measure': 'km', 'weightCatalog': 1, 'icon':...",2020-12-15 19:05:11,5700.0,None,NaT,340639.0,jeep--cherokee-2-800-crd-presao-150-000--2004-...,...,1,2,10,81,1374,None,Cherokee,Dizel,150000,2004
4,AD56823061ME,Podgorica,https://slike.autodiler.me/2020/12/15/AD568230...,"[{'measure': None, 'weightCatalog': 3, 'icon':...",2020-12-15 18:45:47,2000.0,None,NaT,340632.0,mali-potrosac--bez-ulaganja-ad56823061me,...,1,2,10,146,2758,None,Fabia,Benzin,240000,2001
5,AD56883460ME,Bar,https://slike.autodiler.me/2020/12/15/AD568834...,"[{'measure': 'km', 'weightCatalog': 1, 'icon':...",2020-12-15 17:38:39,5500.0,None,NaT,341122.0,prodajem-jeep-cheeroke-u-odlicnom-stanju--ad56...,...,1,1,1,81,1383,None,Liberty,Dizel,179700,2003
6,AD38884239ME,Rožaje,https://slike.autodiler.me/2020/11/22/AD388842...,"[{'measure': 'km', 'weightCatalog': 1, 'icon':...",2020-11-22 15:19:44,1800.0,None,NaT,236153.0,prodajem-auto-u-odlicnom-stanju--prvi-vlasnik-...,...,1,3,21,116,2301,None,206,Benzin,110000,2006
7,AD56820400ME,Svi gradovi - DEU,https://slike.autodiler.me/2020/12/15/AD568204...,"[{'measure': 'km', 'weightCatalog': 1, 'icon':...",2020-12-15 16:49:26,20500.0,None,NaT,340609.0,prodajem-bmw-x5-ad56820400me,...,88,170,1265,24,384,Serija X,X5,Dizel,245000,2012
8,AD56820215ME,Nikšić,https://slike.autodiler.me/2020/12/15/AD568202...,"[{'measure': None, 'weightCatalog': 2, 'icon':...",2020-12-15 16:41:33,1800.0,None,NaT,340607.0,audi-2-4-ad56820215me,...,1,2,9,16,152,None,A6,Benzin+Plin,260000,1997
9,AD56882629ME,Podgorica,https://slike.autodiler.me/2020/12/15/AD568826...,"[{'measure': 'km', 'weightCatalog': 1, 'icon':...",2020-12-15 16:37:39,1400.0,None,NaT,341115.0,prodajem-peugot-206-ad56882629me,...,1,2,10,116,2301,None,206,Benzin,128688,2003
